<a href="https://colab.research.google.com/github/GuanxiongLiu/US-Investment-Toolbox/blob/CallPutFilter/yfinance_CallPutFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install yfinance

In [3]:
import yfinance as yf

# Preparation

In [4]:
import pandas as pd

basic_stocks = pd.read_csv(
    '<path to demo data or your own data>'
)
basic_stocks.head()

,Ticker,BP_1_date,BP_2_date,Vertical_Dis_Per_Channel,Channel_Price
0,SPY,2020-03-23,2021-03-04,4.405,Low
1,QQQ,2020-03-24,2021-05-19,5.457,Low


In [5]:
history_daily = yf.download(# or pdr.get_data_yahoo(...
                            # tickers list or string as well
                            tickers = " ".join(basic_stocks['Ticker'].to_numpy()),

                            # use "period" instead of start/end
                            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                            # (optional, default is '1mo')
                            period = "2y",

                            # fetch data by interval (including intraday if period < 60 days)
                            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                            # (optional, default is '1d')
                            interval = "1d",

                            # group by ticker (to access via data['SPY'])
                            # (optional, default is 'column')
                            group_by = 'ticker',

                            # adjust all OHLC automatically
                            # (optional, default is False)
                            auto_adjust = True,

                            # download pre/post regular market hours data
                            # (optional, default is False)
                            prepost = True,

                            # use threads for mass downloading? (True/False/Integer)
                            # (optional, default is True)
                            threads = True,

                            # proxy URL scheme use use when downloading?
                            # (optional, default is None)
                            proxy = None
                        )

[*********************100%***********************]  2 of 2 completed


In [6]:
dates = [str(d)[0:10] for d in history_daily.index.to_numpy()]      # get all dates and ignore the time
today, dates = dates[-1], dates[:-1]                                # isolate today's date
history_daily = history_daily.drop(index=history_daily.index[-1])   # drop today's data from historical data

In [7]:
history_daily.tail()

SPY              ...         QQQ          
                  Open        High  ...       Close    Volume
Date                                ...                      
2021-05-20  411.799988  416.630005  ...  328.829987  46466100
2021-05-21  416.869995  418.200012  ...  327.010010  50284700
2021-05-24  417.339996  420.320007  ...  332.510010  32475800
2021-05-25  420.329987  420.709991  ...  332.970001  31280300
2021-05-26  418.869995  419.609985  ...  334.130005  25333900

[5 rows x 10 columns]

# Filtering

In [8]:
###
# These two parameters define the interest area (call and put)
# E.g., call_interest_thres = 0.1 defines that call interest zone is an area that the current
# price is higher than any support within 10%
# E.g., put_interest_thres = 0.9 defines that put interest zone is an area that the current
# price is lower than any compression within 10%
###
call_interest_thres, put_interest_thres = 0.3, 0.7

In [9]:
call_direction, put_direction = [], []
for index in range(basic_stocks.shape[0]):
    # get basic info of a specific ticker
    row = basic_stocks.iloc[index]
    # calculate reference point's price
        # index important dates
    BP_1_loc, BP_2_loc, TD_loc = dates.index(row['BP_1_date']), dates.index(row['BP_2_date']), history_daily.shape[0]+1     
        # calculate price diff bt two base points        
    BP_price_diff = (history_daily.iloc[BP_2_loc][(row['Ticker'], row['Channel_Price'])] - history_daily.iloc[BP_1_loc][(row['Ticker'], row['Channel_Price'])])   
        # calculate price diff bt reference point and the second base point (simple math)
    RF_price_diff = (TD_loc - BP_1_loc) / (BP_2_loc - BP_1_loc) * BP_price_diff
        # finalize the price of reference point
    RF_price = history_daily.iloc[BP_1_loc][(row['Ticker'], row['Channel_Price'])] + RF_price_diff
    # get current price of the target ticker
    ticker = yf.Ticker(row['Ticker'])
    price = ticker.history(period='1d', interval='1d').to_numpy()[0,3]
    # calculate the ratio for final determination
    ratio = (abs(price - RF_price) / row['Vertical_Dis_Per_Channel']) - (abs(price - RF_price) // row['Vertical_Dis_Per_Channel'])
    ###
    # assign the ticker to corresponding group based on the following rules
    # (0,call_interest_thres]  --> current price is above a support, [put_interest_thres,1) --> current price is under a pressure)
    ###
    if ratio <= call_interest_thres:
        call_direction.append(row['Ticker'])
    elif ratio >= put_interest_thres:
        put_direction.append(row['Ticker'])
    else:
        pass

In [10]:
print('The CALL direction Tickers are:')
print(call_direction)
print('The PUT direction Tickers are:')
print(put_direction)

The CALL direction Tickers are:
['SPY']
The PUT direction Tickers are:
[]
